In [1]:
import os
import gc
import time
import pickle
import numpy as np
import pandas as pd
import math
from collections import defaultdict

In [ ]:
cache_path = 'fuck_copy4/'
train_path = 'train.csv'
shop_path='shop.csv'
test_path = 'test.csv'
wifi_shop_info=pd.DataFrame()
wifi_shop_max=pd.DataFrame()
wifi_shop_connect_only=pd.DataFrame()
shop_loc=pd.DataFrame()
wifi_shop_count=pd.DataFrame()
flag = True

In [ ]:
def get_time_hour(data):
    data['hourofday']=pd.DatetimeIndex(data.time_stamp).hour
    data['dayofweek']=pd.DatetimeIndex(data.time_stamp).day
    return data
# 相差的分钟数
def diff_of_minutes(time1, time2):
    d = {'5': 0, '6': 31, }
    try:
        days = (d[time1[6]] + int(time1[8:10])) - (d[time2[6]] + int(time2[8:10]))
        try:
            minutes1 = int(time1[11:13]) * 60 + int(time1[14:16])
        except:
            minutes1 = 0
        try:
            minutes2 = int(time2[11:13]) * 60 + int(time2[14:16])
        except:
            minutes2 = 0
        return (days * 1440 - minutes2 + minutes1)
    except:
        return np.nan

# 计算两点之间距离
def call_distance(lat1,lon1,lat2,lon2):
    dx = np.abs(lon1 - lon2)  # 经度差
    dy = np.abs(lat1 - lat2)  # 维度差
    b = (lat1 + lat2) / 2.0
    Lx = 6371004.0 * (dx / 57.2958) * np.cos(b / 57.2958)
    Ly = 6371004.0 * (dy / 57.2958)
    L = (Lx**2 + Ly**2) ** 0.5
    return L


def call_orientation(lat1,lon1,lat2,lon2):
    radLat1 = math.radians(lat1)  
    radLon1 = math.radians(lon1)  
    radLat2 = math.radians(lat2)  
    radLon2 = math.radians(lon2)  
    dLon = radLon2 - radLon1  
    y = math.sin(dLon) * math.cos(radLat2)  
    x = math.cos(radLat1) * math.sin(radLat2) - math.sin(radLat1) * math.cos(radLat2) * math.cos(dLon)  
    brng = math.degrees(math.atan2(y, x))  
    brng = (brng + 360) % 360  
    return brng



def get_label(data):
    true = dict(zip(train['row_id'].values, train['shop_id']))
    print 'label map finish'
    data['label'] = data['row_id'].map(true)
    data['label'] = (data['label'] == data['shop_id']).astype('int')
    return data
def make_wifi_shop_relation(train):
    fuck=train.values
    res1=[]
    res2=[]
    res3=[]
    for i in fuck:
        wifis=i[1].split(';')
        for j in wifis:
            wifi=j.split('|')
            res1.append(i[0])
            res2.append(wifi[0])
            res3.append(int(wifi[1]))
    t_data=pd.DataFrame()
    t_data['shop_id']=res1
    t_data['wifi_id']=res2
    t_data['power']=res3
    return t_data      
def make_wifi_rowid_relation(test):
    fuck=test.values
    res1=[]
    res2=[]
    res3=[]
    for i in fuck:
        wifis=i[1].split(';')
        for j in wifis:
            wifi=j.split('|')
            res1.append(i[0])
            res2.append(wifi[0])
            res3.append(int(wifi[1]))
    t_data=pd.DataFrame()
    t_data['row_id']=res1
    t_data['wifi_id']=res2
    t_data['power']=res3
    return t_data         
def wifi_shop_connect(train):
    fuck=train[['shop_id','wifi_infos']].values
    res1=[]
    res2=[]
    res3=[]
    for i in fuck:
        wifis=i[1].split(';')
        for j in wifis:
            wifi=j.split('|')
            res1.append(i[0])
            res2.append(wifi[0])
            if wifi[2]=='true':
               res3.append(1)
            else:
               res3.append(0)
    t_data=pd.DataFrame()
    t_data['shop_id']=res1
    t_data['wifi_id']=res2
    t_data['connect']=res3
    t_data=t_data[t_data.connect==1]
    return t_data  
def wifi_rowid_conenct(test):
    fuck=test[['row_id','wifi_infos']].values
    res1=[]
    res2=[]
    res3=[]
    for i in fuck:
        wifis=i[1].split(';')
        for j in wifis:
            wifi=j.split('|')
            res1.append(i[0])
            res2.append(wifi[0])
            if wifi[2]=='true':
               res3.append(1)
            else:
               res3.append(0)
    t_data=pd.DataFrame()
    t_data['row_id']=res1
    t_data['wifi_id']=res2
    t_data['connect']=res3
    t_data=t_data[t_data.connect==1]
    return t_data        
####################构造负样本##################
#构造wifishop对应信息
def get_wifi_shop_info(train):
    wifi_train = train[['shop_id','wifi_infos']].drop_duplicates()
    wifi_shop_info=make_wifi_shop_relation(wifi_train)
    wifi_shop_info=wifi_shop_info.groupby(['shop_id','wifi_id'],as_index=False)['power'].agg({'ave_power':'mean'})
    return wifi_shop_info
    
def get_wifi_shop_max(train):
    wifi_train = train[['shop_id','wifi_infos']].drop_duplicates()
    wifi_shop_max=make_wifi_shop_relation(wifi_train)
    wifi_shop_max=wifi_shop_max.groupby(['shop_id','wifi_id'],as_index=False)['power'].agg({'max_power':'max'})
    return wifi_shop_max

def get_wifi_shop_count(train):
    wifi_train = train[['shop_id','wifi_infos']].drop_duplicates()
    wifi_shop_count=make_wifi_shop_relation(wifi_train)
    wifi_shop_count=wifi_shop_count.groupby(['shop_id','wifi_id'],as_index=False)['power'].agg({'wifi inshopcount':'count'})
    wifi_shop_count.sort_values(['shop_id','wifi inshopcount'],inplace=True)
    wifi_shop_count= wifi_shop_count.groupby('shop_id').tail(50)
    return wifi_shop_count
    

# 通过wifi来构建负样本
def get_wifi_shop(train,test):
    result_path = cache_path + 'wifi_shop_%d.hdf' %(train.shape[0]*test.shape[0])
    if os.path.exists(result_path) & flag:
        result = pd.read_hdf(result_path, 'w')
    else:
        #wifi_shop=wifi_shop_count[['shop_id','wifi_id']].drop_duplicates()
        wifi_shop=wifi_shop_info[['shop_id','wifi_id','ave_power']].drop_duplicates()
        #wifi_shop=wifi_shop[wifi_shop['ave_power']>=-80]
       # b=wifi_shop.sort_values('ave_power')
        #wifi_shop= b.groupby('wifi_id').tail(9)
        wifi_test=test[['row_id','wifi_infos']].drop_duplicates()
        wifi_rowid=make_wifi_rowid_relation(wifi_test)
        #选取能量最大的三个看看有几个
        #wifi_rowid=wifi_rowid[wifi_rowid['power']>=-80]
        a=wifi_rowid.sort_values('power')
        wifi_rowid= a.groupby('row_id').tail(7)
        result= pd.merge(wifi_rowid[['row_id','wifi_id']],wifi_shop,on='wifi_id',how='left')
        result=result[['row_id','shop_id']].drop_duplicates()
        result = result[(~result['shop_id'].isnull())]
        result.to_hdf(result_path, 'w', complib='blosc', complevel=5)
    return result
def get_user_end_shop(train,test):
    result_path = cache_path + 'user_end_shop_%d.hdf' %(train.shape[0]*test.shape[0])
    if os.path.exists(result_path) & flag:
        result = pd.read_hdf(result_path, 'w')
    else:
        train_new=pd.merge(train,shop1,on='shop_id',how='left')
        user_shop=train_new[['shop_id','user_id','mall_id']].drop_duplicates()
        result=pd.merge(test[['user_id','mall_id','row_id']],user_shop,on=['user_id','mall_id'])
        result=result[['row_id','shop_id']].drop_duplicates()
        result.to_hdf(result_path, 'w', complib='blosc', complevel=5)
    return result
        
        




#用户特征
# 获取用户历史行为次数
def get_user_count(train,result):
    user_count = train.groupby('user_id',as_index=False)['shop_id'].agg({'user_count':'count'})
    result = pd.merge(result,user_count,on=['user_id'],how='left')
    return result
#hot of shop
def get_shop_hot(train,result):
    shop_hot=train.groupby('shop_id',as_index=False)['row_id'].agg({'shop_hot':'count'})
    result = pd.merge(result,shop_hot,on=['shop_id'],how='left')
    return result

# 获取用户到某商店的次数 
def get_user_shop_count(train, result):
    user_shop_count = train.groupby(['user_id','shop_id'],as_index=False)['user_id'].agg({'user_shop_count':'count'})
    result = pd.merge(result,user_shop_count,on=['user_id','shop_id'],how='left')
    return result
#用户的平均消费能力 最大消费能力 最小消费



#商店特征
#商店属性 商店所属类别，人均消费指数 所属商场号
def get_shop_info(result):
    result=pd.merge(result,shop,on=['shop_id'],how='left')
    result['category_id']=result['category_id'].fillna(0)
    result['category']=result['category_id'].map(lambda x:'0' if x==0 else str(x)[2:]).astype('int')
    return result

   
#用户到某个类型的商店去过几次 是否去过这个类型的商店
def get_uesr_kind(train,result):
    user_kind_count=train.groupby(['user_id','category_id'],as_index=False)['category_id'].agg({'user_kind_count':'count'})
    result=pd.merge(result,user_kind_count,on=['user_id','category_id'],how='left')
    return result

#wifi特征
#有没有连接这个wifi
#这个wifi的
#推荐的店铺和最强 第二强 第三强 第四强wifi的值 现实值 和差值
#强度排名
def f1(x):
    wifipower={}
    wifi=[]
    power=[]
    wifis=x.split(';')
    for i in wifis:
        s=i.split('|')
        wifi.append(s[0])
        power.append(int(s[1]))
    wifipower = dict(zip(wifi, power))
    wifipower=sorted(wifipower.items(),key=lambda x:x[1],reverse=True)
    return wifipower[0]
def f2(x):
    wifipower={}
    wifi=[]
    power=[]
    wifis=x.split(';')
    for i in wifis:
        s=i.split('|')
        wifi.append(s[0])
        power.append(int(s[1]))
    wifipower = dict(zip(wifi, power))
    wifipower=sorted(wifipower.items(),key=lambda x:x[1],reverse=True)
    if(len(wifipower)>1): 
       return wifipower[1]
    else: 
        return 0
 
def get_wifi(result):
    result['wifi1']=result['wifi_infos'].map(f1)
    result['wifi2']=result['wifi_infos'].map(f2)
    result['wifi1_id']=result['wifi1'].map(lambda x:x[0])
    result['wifi1_power']=result['wifi1'].map(lambda x:x[1])
    result['wifi2_id']=result['wifi2'].map(lambda x:0 if x==0 else x[0])
    result['wifi2_power']=result['wifi2'].map(lambda x:0 if x==0 else x[1])
    result['wifi2_power']=result['wifi2'].map(lambda x:0 if x==0 else x[1])
    wifi_shop_info.rename(columns={'wifi_id':'wifi1_id'},inplace=True)
    result=pd.merge(result,wifi_shop_info,on=['shop_id','wifi1_id'],how='left')
    result.rename(columns={'ave_power':'ave_power1'},inplace=True)
    result['ave_power1']=result['ave_power1'].fillna(result['ave_power1'].min())
    result['power_dif1']=abs((result['ave_power1']-result['wifi1_power']).astype(np.float))
    wifi_shop_info.rename(columns={'wifi1_id':'wifi2_id'},inplace=True)
    result=pd.merge(result,wifi_shop_info,on=['shop_id','wifi2_id'],how='left')
    wifi_shop_info.rename(columns={'wifi2_id':'wifi_id'},inplace=True)
    result.rename(columns={'ave_power':'ave_power2'},inplace=True)
    result['ave_power2']=result['ave_power2'].fillna(result['ave_power2'].min())
    result['power_dif2']=abs((result['ave_power2']-result['wifi2_power']).astype(np.float))
    return result

def wifi_name(x):
    wifi=[]
    wifis=x.split(';')
    for i in wifis:
        s=i.split('|')
        wifi.append(s[0])
    return wifi 
def fuck(x):
    x=list(x.split(':'))
    return x
def get_same_relation(result):
    result['w1']=result['wifi_infos'].map(wifi_name)
    t2 = wifi_shop_count.groupby('shop_id')['wifi_id'].agg(lambda x:':'.join(x)).reset_index()
    t2['w2']=t2['wifi_id'].map(fuck)
    del t2['wifi_id']
    result=pd.merge(result,t2,on=['shop_id'],how='left')
    a=list(result['w1'])
    b=list(result['w2'])
    c=[]
    for i in range(len(a)):
        try:
            d=len(set(a[i])&set(b[i]))
            c.append(d)
        except:
            c.append(0)
    result['w3']=c
    #del result['w1']
    del result['w2']
    return result
def get_shop_count_dic(x):
    wificount={}
    wifi=[]
    count=[]
    wifis=x.split('|')
    for i in wifis:
        s=i.split(':')
        wifi.append(s[0])
        count.append(int(s[1]))
    wificount = dict(zip(wifi, count))
    return wificount
def get_wificount(x,y):
    count=0
    try:      
       for i in x:
           if i in y:
              count=count+int(x[i])
           else:
               count=count+0
       return count
    except:
       return 0
def get_wifirat(x, y):
    m=[]
    n=[]
    a={}
    for i in x:
        m.append(i)
        n.append(x[i]/float(y))
    a=dict(zip(m,n))  
    return a   
def wifi_count_intest(result):
    wifi_shop_count['w8']=wifi_shop_count['wifi_id']+':'+wifi_shop_count['wifi inshopcount'].astype('str')
    t1=wifi_shop_count[['shop_id','w8']]
    t2 = t1.groupby('shop_id')['w8'].agg(lambda x:'|'.join(x)).reset_index()
    t2['w8']=t2['w8'].map(get_shop_count_dic)
    del wifi_shop_count['w8']
    result = pd.merge(result,t2,on = ['shop_id'],how = 'left')
    result['wifishopcount'] = map(lambda x, y: get_wificount(x, y) , result['w8'], result['w1']) 
    del result['w8']
    return result
                   
    
def connectamount(result):
    t1=wifi_shop_connect_only[['shop_id','wifi_id']].groupby('shop_id')['wifi_id'].agg(lambda x:':'.join(x)).reset_index()
    t1['h2']=t1['wifi_id'].map(fuck)
    del t1['wifi_id']
    result=pd.merge(result,t1,on=['shop_id'],how='left')
    a=list(result['w1'])
    b=list(result['h2'])
    c=[]
    for i in range(len(a)):
        try:
            d=len(set(a[i])&set(b[i]))
            c.append(d)
        except:
            c.append(0)
    result['h3']=c
    del result['w1']
    del result['h2']
    return result

    
def get_wifi_power(x):
    wifipower={}
    wifi=[]
    power=[]
    wifis=x.split(';')
    for i in wifis:
        s=i.split('|')
        wifi.append(s[0])
        power.append(int(s[1]))
    wifipower = dict(zip(wifi, power))
    return wifipower

def get_shop_wifi_dic(x):
    wifipower={}
    wifi=[]
    power=[]
    wifis=x.split('|')
    for i in wifis:
        s=i.split(':')
        wifi.append(s[0])
        power.append(float(s[1]))
    wifipower = dict(zip(wifi, power))
    return wifipower

def get_knn(x,y):
    sum=0
    for each in x:
        try:
            if each in y:
               sum+=(x[each]-y[each])**2    #历史信息里存在这个信号的话，计算
            else:
               sum += (x[each]-(-90)) ** 2
        except:
            sum=999        
    return sum**0.5*1.0/len(x)

def get_weightpower(x, y):
    sum=0
    for each in x:
        if each in y:
            try:
               sum=sum+float(x[each]*y[each])
            except:
                sum=sum+0
        else:
            sum=sum+0
    return sum

def ger_power_var(result):
    result['w4']=result['wifi_infos'].map(get_wifi_power)
    wifi_shop_info['w']=wifi_shop_info['wifi_id']+':'+wifi_shop_info['ave_power'].astype('str')
    t1=wifi_shop_info[['shop_id','w']]
    t2 = t1.groupby('shop_id')['w'].agg(lambda x:'|'.join(x)).reset_index()
    t2['w']=t2['w'].map(get_shop_wifi_dic)    
    result = pd.merge(result,t2,on = ['shop_id'],how = 'left')
    result['knn_values'] = map(lambda x, y: get_knn(x, y) , result['w4'], result['w'])

    wifi_shop_max['n']=wifi_shop_max['wifi_id']+':'+wifi_shop_max['max_power'].astype('str')
    t1=wifi_shop_max[['shop_id','n']]
    t2 = t1.groupby('shop_id')['n'].agg(lambda x:'|'.join(x)).reset_index()
    t2['n']=t2['n'].map(get_shop_wifi_dic)    
    result = pd.merge(result,t2,on = ['shop_id'],how = 'left')
    result['knn_max_values'] = map(lambda x, y: get_knn(x, y) , result['w4'], result['n'])
    
    del result['w4']
    del result['w']
    del result['n']
    return result
    


#shop_id和当前信号最强的wifi以及第二强的wifi有没有互联记录
def if_wifi_shop_connected(result):
    global wifi_shop_connect_only
    wifi_shop_connect_only.rename(columns={'wifi_id':'wifi1_id','connect':'connect1'},inplace=True)
    result=pd.merge(result,wifi_shop_connect_only,on=['wifi1_id','shop_id'],how='left')
    wifi_shop_connect_only.rename(columns={'wifi1_id':'wifi2_id','connect1':'connect2'},inplace=True)
    result=pd.merge(result,wifi_shop_connect_only,on=['wifi2_id','shop_id'],how='left')
    wifi_shop_connect_only.rename(columns={'wifi2_id':'wifi_id','connect2':'connect'},inplace=True)
    result['connect1']=result['connect1'].fillna(0)
    result['connect2']=result['connect2'].fillna(0)
    result['connect_sum']=result['connect1']+result['connect2']
    return result
#找一下距离差
#加上商店历史上的平均距离，商店位置表中的距离不准
def get_real_shop_loc(result):
    global shop_loc
    shop_loc=shop_loc.groupby('shop_id',as_index=False).agg({'longitude':'mean','latitude':'mean'})
    shop_loc.rename(columns={'longitude':'lon_real','latitude':'lat_real'},inplace=True)
    result=pd.merge(result,shop_loc,on='shop_id',how='left')
    shop_loc.rename(columns={'lon_real':'longitude','lat_real':'latitude'},inplace=True)
    return result

def distance_real_dif(result):
    loc1=list(zip(result.longitude,result.latitude))
    loc2=list(zip(result.lon_real,result.lat_real))
    sum=list(zip(loc1,loc2))
    dis_df1=[]
    for i in sum:
        lat1,lon1=i[0]
        lat2,lon2=i[1]
        d=call_distance(lat1, lon1,lat2, lon2)
        dis_df1.append(d)
    result['real_dis_df']=dis_df1
    return result
def angle_real_dif(result):
    loc1=list(zip(result.longitude,result.latitude))
    loc2=list(zip(result.lon_real,result.lat_real))
    sum=list(zip(loc1,loc2))
    ang_df1=[]
    for i in sum:
        lat1,lon1=i[0]
        lat2,lon2=i[1]
        d=call_orientation(lat1, lon1,lat2, lon2)
        ang_df1.append(d)
    result['real_ang_df']=ang_df1
    return result


def distance_dif(result):
    loc1=list(zip(result.longitude,result.latitude))
    loc2=list(zip(result.longitude_s,result.latitude_s))
    sum=list(zip(loc1,loc2))
    dis_df1=[]
    for i in sum:
        lat1,lon1=i[0]
        lat2,lon2=i[1]
        d=call_distance(lat1, lon1,lat2, lon2)
        dis_df1.append(d)
    result['dis_df']=dis_df1
    return result
#找一下角度差
def angle_dif(result):
    loc1=list(zip(result.longitude,result.latitude))
    loc2=list(zip(result.longitude_s,result.latitude_s))
    sum=list(zip(loc1,loc2))
    ang_df1=[]
    for i in sum:
        lat1,lon1=i[0]
        lat2,lon2=i[1]
        d=call_orientation(lat1, lon1,lat2, lon2)
        ang_df1.append(d)
    result['ang_df']=ang_df1
    return result

#行为发生时间
def get_time(train,result):
    result=get_time_hour(result)
    train=get_time_hour(train)
    t1=train.groupby('shop_id',as_index=False)['hourofday'].agg({'h_s_m':'mean','h_s_v':'std','h_s':'median'})
    result=pd.merge(result,t1,on='shop_id',how='left')
    result['time_dif']=abs(result['h_s']-result['hourofday'])
    return result